# Download libraries

In [25]:
!pip install tokenizers==0.21.0

# Import libraries

In [26]:
import math
import os
import re
import time
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordLevelTrainer

# Load and read dataset

In [27]:
DATASET_PATH = 'that_ngon_tu_tuyet_final.csv'

df = pd.read_csv(DATASET_PATH)
df

,content,url
0,"Giàu làm chị, khó làm em, Sang chớ kiêu căng, ...",https://www.thivien.net/Nguy%E1%BB%85n-B%E1%BB...
1,"Nhất bộc rành rành lại thập thành, Ở cho thực ...",https://www.thivien.net/Tr%E1%BB%8Bnh-Doanh/Ba...
2,"Văn chương phú lục đã xong rồi, Thừa giấy làm ...",https://www.thivien.net/Nguy%E1%BB%85n-Qu%E1%B...
3,"Đồn Phố Hiến vui hơn kinh kỳ, Chơi ba ngày chẳ...",https://www.thivien.net/Nguy%E1%BB%85n-Qu%E1%B...
4,"Quan nhậm châu khách Hoan châu, Cầm bằng mà cũ...",https://www.thivien.net/Khuy%E1%BA%BFt-danh-Vi...
...,...,...
285,Từng đêm xuân cháy từng biển mộng Mênh mông ru...,https://www.thivien.net/H%C3%A0n-Qu%E1%BB%91c-...
286,Người ngồi như cỏ hoang vu quá! Phố chết tưng ...,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...
287,Chỉ giống người trên mặt địa cầu Đang tâm hành...,https://www.thivien.net/V%C5%A9-Ho%C3%A0ng-Ch%...
288,Hồ đã tan thây sóng Bạch Đằng Minh còn mất vía...,https://www.thivien.net/V%C5%A9-Ho%C3%A0ng-Ch%...


In [28]:
df['content'][0].split('\n')

['Giàu làm chị, khó làm em, Sang chớ kiêu căng, khó chớ hiềm . Dưới biết kính trên, trên dấu dưới, Ấy nhà còn thịnh, phúc còn thêm.']

#  Build vectorization function

In [29]:
def text_normalize(text):
    text = text.strip()
    return text

df['content'] = df['content'].apply(lambda x: text_normalize(x))

In [30]:
for idx, row in df.iterrows():
    print(f'{idx+1}.')
    print(row['content'])
    print()

    if idx == 10:
        break

1.
Giàu làm chị, khó làm em, Sang chớ kiêu căng, khó chớ hiềm . Dưới biết kính trên, trên dấu dưới, Ấy nhà còn thịnh, phúc còn thêm.

2.
Nhất bộc rành rành lại thập thành, Ở cho thực mặc ấy là ngoan. Đầy vơi chớ chớ chiều lòng thế, Thì mới nên danh giá tao đàn.

3.
Văn chương phú lục đã xong rồi, Thừa giấy làm chi chẳng vẽ voi? Nhắn nhủ một lời cho chúng biết: Đứa nào cười tớ nó ăn bòi.

4.
Đồn Phố Hiến vui hơn kinh kỳ, Chơi ba ngày chẳng thấy quái gì. Ngô lớn, Ngô con răng trắng nhởn, Đĩ già, đĩ trẻ đách thâm sì.

5.
Quan nhậm châu khách Hoan châu, Cầm bằng mà cũng chẳng nên rầu. Nay phảng phất thông tin sứ, Vò thấu chung tình nỗi nhỏ to.

6.
Từ ghé non Bồng diễn bạn tiên, Chạnh lòng khao khát để riêng phiền. Trình từ diệu vợi người thân hữu, Thề có khi nào lặng chốc quên.

7.
Thấy dân rét mướt, nghĩ mà thương, Vậy phải lên ngôi gỡ mối giường . Tay ngọc lần đưa thoi nhật nguyệt , Gót vàng dận dạn máy âm dương .

8.
Xem ý trời đà ấy dục tình, Ngại vì mưa lớn mới thanh minh. Sương nghiê

In [31]:
def text_generator():
    for text in df['content']:
        yield text

UNK_TOKEN = "[UNK]"
PAD_TOKEN = "[PAD]"
EOS_TOKEN = "[EOS]"
SOS_TOKEN = "[SOS]"
EOL_TOKEN = "[EOL]"

tokenizer = Tokenizer(WordLevel(unk_token=UNK_TOKEN))
tokenizer.pre_tokenizer = Whitespace()

trainer = WordLevelTrainer(special_tokens=[UNK_TOKEN, PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, EOL_TOKEN])
tokenizer.train_from_iterator(text_generator(), trainer=trainer)

In [32]:
vocab = tokenizer.get_vocab()
vocab_size = len(vocab)
print("Vocab size:", vocab_size)
print("Vocabulary:")
vocab

Vocab size: 2537
Vocabulary:


{'tiên': 250,
 'sư': 854,
 'lộ': 1220,
 'khứng': 2024,
 'kiểm': 2028,
 'thoáng': 865,
 'Thổi': 1733,
 'ích': 2454,
 'dây': 1128,
 'luỹ': 2048,
 'làng': 435,
 'mạnh': 2106,
 'Chẳng': 194,
 'tổng': 1390,
 'đắng': 1441,
 'giật': 1151,
 'hời': 1993,
 'Không': 221,
 'kìa': 787,
 'chín': 229,
 'Đạt': 1424,
 'Ngỏ': 1652,
 'thuỳ': 1333,
 'tất': 2377,
 'tử': 144,
 'rờm': 2269,
 'oan': 1287,
 'Nhẹ': 1658,
 'giống': 424,
 'rừng': 213,
 'Sức': 1708,
 'Quả': 1680,
 'đa': 2485,
 'Đầu': 641,
 'thai': 860,
 'Lưu': 1605,
 'bướm': 408,
 'mộng': 97,
 'Lại': 311,
 'hở': 1995,
 'ghế': 423,
 'cù': 746,
 'chim': 320,
 'Mả': 1636,
 'Giêng': 1535,
 'Công': 497,
 'Môi': 998,
 'Trút': 404,
 'Miền': 1624,
 'năn': 2181,
 '”': 131,
 'chạp': 1869,
 'lắc': 2076,
 'Na': 1006,
 'Hổ': 1565,
 'Bồi': 1477,
 'Buồn': 1461,
 'quỳ': 2235,
 'Ở': 1451,
 'II': 395,
 'nấu': 1278,
 'nhau': 54,
 'khảo': 1185,
 'Thày': 1717,
 'vậy': 293,
 'Biển': 934,
 'Gian': 968,
 'ruộng': 1307,
 'Chợt': 1500,
 'Duyên': 661,
 'giương': 1944,
 'khu

In [33]:
default_index = tokenizer.token_to_id("[UNK]")
print("Default index for unknown tokens:", default_index)

Default index for unknown tokens: 0


In [34]:
PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)
EOS_TOKEN_ID = tokenizer.token_to_id(EOS_TOKEN)

MAX_SEQ_LEN = 35
tokenizer.enable_padding(length=MAX_SEQ_LEN,
                         pad_id=PAD_TOKEN_ID,
                         pad_token=PAD_TOKEN)
tokenizer.enable_truncation(max_length=MAX_SEQ_LEN)

In [35]:
test_text = df['content'][0].split('\n')[0]
test_encoded = tokenizer.encode(df['content'][0].split('\n')[0])

print("Token IDs:", test_encoded.ids)
print("Tokens:", test_encoded.tokens)
print("Attention Mask:", test_encoded.attention_mask)

Token IDs: [1534, 44, 324, 5, 135, 44, 57, 5, 1032, 175, 1188, 1115, 5, 135, 175, 1163, 6, 498, 26, 2032, 123, 5, 123, 330, 1131, 5, 929, 118, 17, 1344, 5, 1296, 17, 463, 6]
Tokens: ['Giàu', 'làm', 'chị', ',', 'khó', 'làm', 'em', ',', 'Sang', 'chớ', 'kiêu', 'căng', ',', 'khó', 'chớ', 'hiềm', '.', 'Dưới', 'biết', 'kính', 'trên', ',', 'trên', 'dấu', 'dưới', ',', 'Ấy', 'nhà', 'còn', 'thịnh', ',', 'phúc', 'còn', 'thêm', '.']
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [36]:
test_decoded = tokenizer.decode(test_encoded.ids)
print("Decoded text:", test_decoded)

Decoded text: Giàu làm chị , khó làm em , Sang chớ kiêu căng , khó chớ hiềm . Dưới biết kính trên , trên dấu dưới , Ấy nhà còn thịnh , phúc còn thêm .


# Create pytorch dataset

In [97]:
class PoemDataset(Dataset):
    def __init__(self, df, tokenizer, max_seq_len):
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.input_seqs, self.target_seqs, self.attn_masks = self.create_samples(df)

    def __len__(self):
        return len(self.input_seqs)

    def __getitem__(self, idx):
        return self.input_seqs[idx], self.target_seqs[idx], self.attn_masks[idx]

    def split_content(self, content):

        content = content.strip()

        raw_tokens = re.split(r'\s+', content)

        tokens = []
        for token in raw_tokens:
            if token and not any(ch.isalpha() for ch in token):
                if tokens:
                    tokens[-1] = tokens[-1] + token
                else:
                    tokens.append(token)
            else:
                tokens.append(token)

        if len(tokens) != 28:
          return []

        lines = []

        for i in range(0, 28, 7):
          line = " ".join(tokens[i:i+7])
          lines.append(line)

        return [lines]

    def create_samples(self, df):
        all_input_seqs = []
        all_target_seqs = []
        all_attn_masks = []

        for _, row in df.iterrows():
            content = row['content']
            samples = self.split_content(content)
            for sample in samples:
                sample_inputs, sample_targets, sample_attn = self.prepare_sample(sample)
                all_input_seqs.extend(sample_inputs)
                all_target_seqs.extend(sample_targets)
                all_attn_masks.extend(sample_attn)

        all_input_seqs = torch.tensor(all_input_seqs, dtype=torch.long)
        all_target_seqs = torch.tensor(all_target_seqs, dtype=torch.long)
        all_attn_masks = torch.tensor(all_attn_masks, dtype=torch.float)

        return all_input_seqs, all_target_seqs, all_attn_masks

    def prepare_sample(self, sample):
        input_seqs = []
        target_seqs = []
        attn_masks = []

        input_text = "[SOS] " + " [EOL] ".join(sample) + " [EOL] [EOS]"

        unpadded_encoding = self.tokenizer.encode(input_text)
        input_ids = unpadded_encoding.ids

        for idx in range(1, len(input_ids)):
            prefix_ids = input_ids[:idx]
            prefix_text = self.tokenizer.decode(prefix_ids, skip_special_tokens=False)
            prefix_encoding = self.tokenizer.encode(prefix_text)

            target_ids = input_ids[1:idx+1]
            target_text = self.tokenizer.decode(target_ids, skip_special_tokens=False)
            target_encoding = self.tokenizer.encode(target_text)

            input_seqs.append(prefix_encoding.ids)
            target_seqs.append(target_encoding.ids)
            attn_masks.append(prefix_encoding.attention_mask)

        return input_seqs, target_seqs, attn_masks

In [98]:
TRAIN_BS = 16
train_dataset = PoemDataset(
    df=df,
    tokenizer=tokenizer,
    max_seq_len=MAX_SEQ_LEN
)

train_loader = DataLoader(
    train_dataset,
    batch_size=TRAIN_BS,
    shuffle=False
)

In [99]:
input_seqs, target_seqs, attn_masks = next(iter(train_loader))

print(input_seqs[0])
print(target_seqs[0])
print(attn_masks[0])

tensor([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([1656,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [100]:
for idx in range(len(input_seqs)):
    print(tokenizer.decode(input_seqs[idx].tolist(), skip_special_tokens=False))
    print(tokenizer.decode(target_seqs[idx].tolist(), skip_special_tokens=False))

[SOS] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Nhất [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[SOS] Nhất [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Nhất bộc [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[SOS] Nhất bộc [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

# Create model

In [101]:
class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dims, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dims, 2) * (-math.log(10000.0) / embedding_dims))
        pe = torch.zeros(max_len, 1, embedding_dims)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        x = self.dropout(x)
        return x

In [102]:
class TransformerModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dims,
        n_heads,
        hidden_dims,
        n_layers,
        dropout=0.5
    ):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.embedding = nn.Embedding(vocab_size, embedding_dims)
        self.embedding_dims = embedding_dims

        self.pos_encoder = PositionalEncoding(embedding_dims, dropout)
        encoder_layers = nn.TransformerEncoderLayer(
            embedding_dims,
            n_heads,
            hidden_dims,
            dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        self.linear = nn.Linear(embedding_dims, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask=None, attn_masks=None):
        src = self.embedding(src) * math.sqrt(self.embedding_dims)
        src = self.pos_encoder(src)

        if src_mask is None:
            src_mask = nn.Transformer.generate_square_subsequent_mask(src.size(0)).to(src.device)

        output = self.transformer_encoder(src, mask=src_mask, src_key_padding_mask=attn_masks)
        output = self.linear(output)

        return output

In [103]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIMS = 128
HIDDEN_DIMS = 64
N_LAYERS = 1
N_HEADS = 16
DROPOUT = 0.2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_tests = torch.randint(1, 10, (1, 5)).to(device)

model = TransformerModel(
    VOCAB_SIZE,
    EMBEDDING_DIMS,
    N_HEADS,
    HIDDEN_DIMS,
    N_LAYERS,
    DROPOUT
).to(device)

with torch.no_grad():
    output = model(input_tests)
    print(output.shape)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


torch.Size([1, 5, 2537])


# Training

In [114]:
LR = 0.1
EPOCHS = 100

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

In [115]:
model.train()
for epoch in range(EPOCHS):
    losses = []
    for idx, samples in enumerate(train_loader):
        input_seqs, target_seqs, attn_masks = samples
        input_seqs = input_seqs.to(device)
        target_seqs = target_seqs.to(device)
        attn_masks = attn_masks.to(device).permute(1, 0)

        output = model(input_seqs, attn_masks=attn_masks)
        output = output.permute(0, 2, 1)
        loss = criterion(output, target_seqs)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        losses.append(loss.item())

    total_loss = sum(losses) / len(losses)
    print(f'EPOCH {epoch+1} - Loss {total_loss}')
    scheduler.step()

EPOCH 1 - Loss 1.013584109540427
EPOCH 2 - Loss 1.0277378571011844
EPOCH 3 - Loss 1.0275616918414994
EPOCH 4 - Loss 1.026461211501362
EPOCH 5 - Loss 1.0271084958758414
EPOCH 6 - Loss 1.0264566526655277
EPOCH 7 - Loss 1.0256429821415085
EPOCH 8 - Loss 1.0252498176266625
EPOCH 9 - Loss 1.023189600712591
EPOCH 10 - Loss 1.0231688692760121
EPOCH 11 - Loss 1.0228756141532787
EPOCH 12 - Loss 1.021983025673297
EPOCH 13 - Loss 1.0201628499152224
EPOCH 14 - Loss 1.0215672448737219
EPOCH 15 - Loss 1.018059651020867
EPOCH 16 - Loss 1.0178794020316995
EPOCH 17 - Loss 1.018328448713583
EPOCH 18 - Loss 1.0158064386481165
EPOCH 19 - Loss 1.0143071852773158
EPOCH 20 - Loss 1.0134363514109228
EPOCH 21 - Loss 1.013066630830782
EPOCH 22 - Loss 1.0136038142302075
EPOCH 23 - Loss 1.0109836082168586


KeyboardInterrupt: 

# Inference

In [116]:
def greedy_sampling(model, tokenizer, input_text, MAX_GENERATION_LEN):
    tokenizer.no_padding()
    tokenizer.no_truncation()
    input_encoded = tokenizer.encode(input_text)
    input_ids = input_encoded.ids
    eos_token_id = tokenizer.token_to_id(EOS_TOKEN)

    generated_ids = input_ids.copy()

    for _ in range(MAX_GENERATION_LEN - len(input_ids)):
        input_tensor = torch.tensor([generated_ids], dtype=torch.long).to(device)
        with torch.no_grad():
            outputs = model(input_tensor)

        next_token_id = torch.argmax(outputs[0, -1, :], dim=-1).item()
        generated_ids.append(next_token_id)

        if next_token_id == eos_token_id:
            break

    return tokenizer.decode(generated_ids, skip_special_tokens=False)

In [117]:
model.eval()
input_text = "[SOS] Động"
MAX_GENERATION_LEN = 50

generated_text = greedy_sampling(model, tokenizer, input_text, MAX_GENERATION_LEN)
generated_text = generated_text.replace(SOS_TOKEN, '').replace(EOS_TOKEN, '')
lines = generated_text.split(EOL_TOKEN)

for line in lines:
    print(''.join(line))

 Động người 
 Bóng giặc hung tàn 
 Bóng giặc hung tàn 
 Bóng giặc hung tàn 
 Bóng giặc hung tàn 
 Bóng giặc hung tàn 
 Bóng giặc hung tàn 
 Bóng giặc hung tàn 
 Bóng giặc hung tàn 
 Bóng giặc hung tàn 
 Bóng


In [118]:
def beams_search(beams, model, device, beam_width):
    candidates = []

    for score, seq in beams:
        if seq[-1] == eos_token_id:
            candidates.append((score, seq))
            continue

        input_tensor = torch.tensor([seq], dtype=torch.long).to(device)
        with torch.no_grad():
            outputs = model(input_tensor)

        log_probs = torch.log_softmax(outputs[0, -1, :], dim=-1)
        top_scores, top_indices = torch.topk(log_probs, beam_width)

        for i in range(beam_width):
            new_seq = seq + [top_indices[i].item()]
            new_score = score + top_scores[i].item()
            candidates.append((new_score, new_seq))

    return sorted(candidates, key=lambda x: x[0], reverse=True)[:beam_width]

In [119]:
model.eval()
input_text = '[SOS] Động'
beam_width = 5
MAX_GENERATION_LEN = 50

tokenizer.no_padding()
tokenizer.no_truncation()
input_encoded = tokenizer.encode(input_text)
input_ids = input_encoded.ids
eos_token_id = tokenizer.token_to_id(EOS_TOKEN)

beams = [(0.0, input_ids)]

for _ in range(MAX_GENERATION_LEN):
    beams = beams_search(beams, model, device, beam_width)

    if all(seq[-1] == eos_token_id for _, seq in beams):
      break

best_seq = beams[0][1]
generated_text = tokenizer.decode(best_seq, skip_special_tokens=False)
generated_text = generated_text.replace(SOS_TOKEN, '').replace(EOS_TOKEN, '')
lines = generated_text.split(EOL_TOKEN)

for line in lines:
    print(''.join(line))

 Động người đi ngàn liễu khóc vì sao nói khoa này sắp đổi Thương vua mến chúa phải tù , 
 Bóng giặc hung tàn nanh quỷ dữ 
 Bóng giặc hung tàn nanh quỷ dữ 
 Bóng giặc hung tàn nanh quỷ dữ 
 Bóng giặc hung tàn nanh quỷ


In [120]:
def sampling(logits):
  probabilites = F.softmax(logits, dim=-1)
  next_token_id = torch.multinomial(probabilites, 1).item()
  return next_token_id

In [131]:
model.eval()
input_text = '[SOS] Động'
input_tokens = tokenizer.encode(input_text).tokens
input_ids = [vocab[token] for token in input_tokens]
eos_token_id = vocab[EOS_TOKEN]
generated_ids = input_ids.copy()
MAX_GENERATION_LEN = 50

for _ in range(MAX_GENERATION_LEN):
    input_tensor = torch.tensor([generated_ids], dtype=torch.long).to(device)
    with torch.no_grad():
        output = model(input_tensor)

    last_token_logits = output[0, -1, :]
    next_token_id = sampling(last_token_logits)
    generated_ids.append(next_token_id)

    if next_token_id == eos_token_id:
        break

# Convert the generated tokens back to text
generated_text = tokenizer.decode(generated_ids, skip_special_tokens=False)
generated_text = generated_text.replace(SOS_TOKEN, '').replace(EOS_TOKEN, '')
lines = generated_text.split(EOL_TOKEN)

for line in lines:
    print(''.join(line))

 Động người thương một ngày bay 
 Bóng ai ? 
 Hình bóng vóc người 
 Công nghiệp quá tuồng ! 
 Thiên lý nhân 
 Tình nhạc trăng lặn mất tăm , non Bồng diễn bạn bầu rượu cưới 
 Non xanh đúc một chút phấn thông . 
 Bóng


# Save model

In [126]:
torch.save(model.state_dict(), "poem_generation_from_scratch_weights.pth")